# **28.09.23**

# **Exercise of Data Linkage Concepts and Techniques**

---


## **1. Data Preparation**

In [19]:
import pandas as pd
import numpy as np
from math import floor, ceil

import matplotlib.pyplot as plt

In [20]:
# Actually the data is come from febrl4 in recordlinkage toolkit
# from recordlinkage.datasets import load_febrl4
dfA = pd.read_csv('dataA.csv', index_col=0)
dfB = pd.read_csv('dataB.csv', index_col=0)

you can check the data at your convenience 

## **2. Calculation of Jaro-Winkler Distance**

Define functions to calculate the Jaro-Winkler Distance

In [21]:
# Python3 implementation of JW di
 
# Function to calculate the
# Jaro Similarity of two s
def jaro_distance(s1, s2):
     
    # If the s are equal
    if (s1 == s2):
        return 1.0
 
    # Length of two s
    len1 = len(s1)
    len2 = len(s2)
 
    # Maximum distance upto which matching
    # is allowed
    max_dist = floor(max(len1, len2) / 2) - 1
 
    # Count of matches
    match = 0
 
    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first
    for i in range(len1):
 
        # Check if there is any matches
        for j in range(max(0, i - max_dist),
                       min(len2, i + max_dist + 1)):
             
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
 
    # If there is no match
    if (match == 0):
        return 0.0
 
    # Number of transpositions
    t = 0
    point = 0
 
    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1):
        if (hash_s1[i]):
 
            # Find the next matched character
            # in second
            while (hash_s2[point] == 0):
                point += 1
 
            if (s1[i] != s2[point]):
                t += 1
            point += 1
    t = t//2
 
    # Return the Jaro Similarity
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0
 
def jaro_winkler_distance(s1, s2):
    jaro_dist = jaro_distance(s1, s2)
    
    # Length of common prefix
    L = 0
    for l1, l2 in zip(s1, s2):
        if l1 == l2:
            L += 1
        else:
            break
    L = max(4, L)  # Take at most 4 characters
    p = 0.1  # Scaling factor
    
    return jaro_dist + (L * p * (1 - jaro_dist))
def compute_similarity_matrix(dfA, dfB):
    n, m = dfA.shape
    p, _ = dfB.shape
    
    # Create an empty matrix to store the results
    results = []
    
    # Iterate through each pair of records
    for idx_a, row_a in dfA.iterrows():
        for idx_b, row_b in dfB.iterrows():
            # For each variable, compute the Jaro-Winkler similarity
            similarities = [jaro_winkler_distance(str(row_a.iloc[i]), str(row_b.iloc[i])) for i in range(m)]
            results.append(similarities)
            
    # Convert to DataFrame with combined index
    index_pairs = [(idx_a, idx_b) for idx_a in dfA.index for idx_b in dfB.index]
    df_result = pd.DataFrame(results, columns=dfA.columns, index=pd.MultiIndex.from_tuples(index_pairs))
    
    return df_result

The function compute_similarity_matrix() will take two datasets and compute Jaro-Winkler Distance for all pairs and all attributes 

### **Exercise #1**

+ *What is meaning of the Jaro-Winkler Distance?*
+ *Calculate the Jaro-Winkler Distance as df_similarity.*

In [22]:
# Your solutions here
df_similarity = compute_similarity_matrix(dfA, dfB)
df_similarity.head(100)

given_name   surname  address_1  address_2
rec-2670-org rec-1308-dup-0    0.400000  0.400000   0.748556   0.753077
             rec-4803-dup-0    0.724444  0.683333   0.742757   0.677143
             rec-4357-dup-0    0.760000  0.678571   0.735152   0.753333
             rec-876-dup-0     0.400000  0.650000   0.756061   0.753333
             rec-3085-dup-0    0.673333  0.400000   0.698812   0.686667
...                                 ...       ...        ...        ...
             rec-1859-dup-0    0.400000  0.675000   0.773766   0.753333
             rec-2548-dup-0    0.673333  0.683333   0.672121   0.705000
             rec-3769-dup-0    0.400000  0.400000   0.801136   0.695000
             rec-45-dup-0      0.780000  0.678571   0.754545   0.719286
             rec-2880-dup-0    0.730000  0.683333   0.720804   0.776667

[100 rows x 4 columns]

### **Exercise #2**

+ *If our similarity levels $L_k$ is 3 (different, similar, identical),*
+ *convert the result into similarity levels based on the threshold values of 0.88 and 0.94*

In [23]:
def convert_matrix_values(matrix):
    """
    Convert matrix values based on the criteria.
    
    Parameters:
        matrix (pd.DataFrame): Input matrix to convert.

    Returns:
        pd.DataFrame: Matrix with converted values.
    """
    
    # Create conditions for each category
    conditions = [
        (matrix < 0.88),
        ((matrix >= 0.88) & (matrix < 0.94)),
        ((matrix >= 0.94) & (matrix <= 1)),
        (matrix > 1),
        (matrix.isna())
    ]

    # Corresponding values for each condition
    values = ['different', 'similar', 'identical', 'Error', np.nan]

    # Apply conditions and set values
    matrix_converted = pd.DataFrame(np.select(conditions, values, default=np.nan), index=matrix.index, columns=matrix.columns)

    return matrix_converted

converted_df = convert_matrix_values(df_similarity)
converted_df.head(100)

given_name    surname  address_1  address_2
rec-2670-org rec-1308-dup-0  different  different  different  different
             rec-4803-dup-0  different  different  different  different
             rec-4357-dup-0  different  different  different  different
             rec-876-dup-0   different  different  different  different
             rec-3085-dup-0  different  different  different  different
...                                ...        ...        ...        ...
             rec-1859-dup-0  different  different  different  different
             rec-2548-dup-0  different  different  different  different
             rec-3769-dup-0  different  different  different  different
             rec-45-dup-0    different  different  different  different
             rec-2880-dup-0  different  different  different  different

[100 rows x 4 columns]

### **Exercise #3**

+ *If we set the rule that a pair is considered as a match if they have no "different", then*
+ *Identify the matched pairs and compute number of matched pairs*

In [24]:
# Create a boolean DataFrame where 'identical' is True
identical_mask = (converted_df == 'identical')

# Create a boolean DataFrame where 'similar' is True
similar_mask = (converted_df == 'similar')

# Create a boolean DataFrame where 'different' is True
different_mask = (converted_df == 'different')

# Count the number of 'identical', 'similar', and 'different' values per row
identical_counts = identical_mask.sum(axis=1)
similar_counts = similar_mask.sum(axis=1)
different_counts = different_mask.sum(axis=1)

# Filter the rows where either there are two 'identical' values, 
# three 'similar' values, and no 'different' values
filtered_df = converted_df[(different_counts == 0)]

print("Pairs with either two 'identical' attributes or three 'similar' attributes, and no 'different' attributes:")
print(filtered_df)

print("\nNumber of such pairs:", len(filtered_df))


Pairs with either two 'identical' attributes or three 'similar' attributes, and no 'different' attributes:
                            given_name    surname  address_1  address_2
rec-4275-org rec-4275-dup-0  identical  identical  identical  identical
rec-3813-org rec-3813-dup-0  identical  identical  identical      Error
rec-4500-org rec-4500-dup-0  identical  identical  identical  identical
rec-4478-org rec-4478-dup-0  identical    similar  identical  identical
rec-4818-org rec-4818-dup-0  identical  identical  identical  identical
rec-4365-org rec-4365-dup-0  identical  identical  identical  identical

Number of such pairs: 6


In [25]:
# Extract indices for dfA and dfB
dfA_indices = [index[0] for index in filtered_df.index]
dfB_indices = [index[1] for index in filtered_df.index]

# Print rows from dfA
print("Rows from dfA:")
print(dfA.loc[dfA_indices])

# Print rows from dfB
print("\nRows from dfB:")
print(dfB.loc[dfB_indices])

Rows from dfA:
             given_name       surname          address_1           address_2
rec_id                                                                      
rec-4275-org     isobel       burnell  heidelberg street  cinema centre bldg
rec-3813-org        amy         mason   casuarina street   memorial hospital
rec-4500-org       zali      mcgregor    goulburn street                 NaN
rec-4478-org       joel          reid     pearson street              lytton
rec-4818-org  alexandra    yiannoulis    archdall street          tall pines
rec-4365-org    georgia  glover-smith       howse street                 NaN

Rows from dfB:
               given_name       surname           address_1  \
rec_id                                                        
rec-4275-dup-0     isobel       burnell  heidelb erg street   
rec-3813-dup-0        amy         masno    casuarina street   
rec-4500-dup-0       zali      mcgregor     goulburn street   
rec-4478-dup-0       joel          rei

## **3. Using Fellegi-Sunter method with ECM through Record linkage Toolkit (optional)**

Installation of Python Record linkage Toolkit

The Python Record linkage Toolkit requires Python 3.6 or higher. 

Install the package easily with pip

$ pip install recordlinkage

The related website is https://recordlinkage.readthedocs.io/en/latest/installation.html

You can check about the installation using the following code

In [26]:
import recordlinkage as rl

### **Exercise #4**

+ *Try to run the following code to compute the result using the original JW scores*
+ *Try to understand the result*

In [27]:
cl = rl.ECMClassifier(binarize=0.8)
cl.fit(df_similarity)
fsweights = cl.log_weights

In [28]:
print("p probability P(Match):", cl.p)
print("log weights of features:", fsweights)

p probability P(Match): 0.0002413053563130277
log weights of features: {'given_name': {0.0: -2.7585043551944866, 1.0: 3.1367565452246335}, 'surname': {0.0: -2.08044937493793, 1.0: 3.509705415463097}, 'address_1': {0.0: -1.3614644853598485, 1.0: 1.5977437413052415}, 'address_2': {0.0: -2.776119972690446, 1.0: 3.330966879534355}}


### **Exercise #5**

+ *Using the match weight to compute the link probability of each pairs*
+ *Identify the matched pairs with a threshold of 0.9*
+ *Will this result better?*

In [29]:
# Extract the weights for each attribute
weights = {key: value.get(1.0, 0.0001) for key, value in fsweights.items()}

# Normalize the weights so they sum to 1
total_weight = sum(weights.values())
normalized_weights = {key: value / total_weight for key, value in weights.items()}

# Multiply each column by its corresponding normalized weight
weighted_df = df_similarity.multiply(normalized_weights, axis=1)

# Sum across the columns for each row
weighted_sum = weighted_df.sum(axis=1)

# Filter the pairs with values higher than 0.7
filtered_pairs2 = df_similarity[weighted_sum > 0.9]

# Print the results
print("Pairs with values higher than 0.9:")
print(filtered_pairs2.index)

print("\nNumber of such pairs:", len(filtered_pairs2))

Pairs with values higher than 0.9:
MultiIndex([('rec-4058-org', 'rec-3685-dup-0'),
            ('rec-4275-org', 'rec-4275-dup-0'),
            ('rec-3813-org', 'rec-3813-dup-0'),
            ('rec-4500-org', 'rec-4500-dup-0'),
            ('rec-4478-org', 'rec-4478-dup-0'),
            ('rec-4818-org', 'rec-4818-dup-0'),
            ( 'rec-468-org',  'rec-468-dup-0'),
            ('rec-4365-org', 'rec-4365-dup-0'),
            ('rec-4217-org', 'rec-1132-dup-0')],
           )

Number of such pairs: 9


In [30]:
# Extract indices for dfA and dfB
dfA_indices = [index[0] for index in filtered_pairs2.index]
dfB_indices = [index[1] for index in filtered_pairs2.index]

# Print rows from dfA
print("Rows from dfA:")
print(dfA.loc[dfA_indices])

# Print rows from dfB
print("\nRows from dfB:")
print(dfB.loc[dfB_indices])


Rows from dfA:
             given_name       surname           address_1           address_2
rec_id                                                                       
rec-4058-org   benjamin       coleman    canopus crescent                 NaN
rec-4275-org     isobel       burnell   heidelberg street  cinema centre bldg
rec-3813-org        amy         mason    casuarina street   memorial hospital
rec-4500-org       zali      mcgregor     goulburn street                 NaN
rec-4478-org       joel          reid      pearson street              lytton
rec-4818-org  alexandra    yiannoulis     archdall street          tall pines
rec-468-org      connor          arda       fenner street          summervale
rec-4365-org    georgia  glover-smith        howse street                 NaN
rec-4217-org    barnaby         dixon  charteris crescent                 NaN

Rows from dfB:
               given_name       surname           address_1  \
rec_id                                          